### Import data

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv("labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

In [3]:
train.shape

(25000, 3)

In [4]:
train.columns.values

array(['id', 'sentiment', 'review'], dtype=object)

### Parse data

In [5]:
from bs4 import BeautifulSoup

In [6]:
# Initialize the BeautifulSoup object on a single movie review     
example1 = BeautifulSoup(train['review'][0], "lxml")

In [7]:
example1.get_text()

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 2

### Remove punctuations and numbers

In [8]:
import re

In [9]:
# Use regular expressions to do a find-and-replace
letters_only = re.sub("[^a-zA-Z]",           # The pattern to search for
                      " ",                   # The pattern to replace it with
                      example1.get_text() )  # The text to search
print(letters_only)

 With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    mi

In [10]:
lower_case = letters_only.lower()

In [11]:
words = lower_case.split()

### NLTK

In [12]:
import nltk

In [13]:
from nltk.corpus import stopwords # Import the stop word list
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

#### Function to apply to all

In [14]:
def review_to_words(review):
    
    # Read HTML
    review_text = BeautifulSoup(review, 'lxml').get_text()
    
    # Keep only letters (replace anything which is not a letter)
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    
    # Convert to lower case and split into words
    words = letters_only.lower().split()
    
    # Remove stop words
    stops = set(stopwords.words('english'))
    
    meaningful_words = [word for word in words if word not in stops]
    
    # Rejoin
    return " ".join(meaningful_words)

In [15]:
review_to_words(train['review'][0])

'stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate workin

### Create clean dataset

In [16]:
import sys

In [17]:
# Get the number of reviews based on the dataframe column size
num_reviews = train["review"].size

In [18]:
sys.stdout.write('Cleaning and parsing the training set movie reviews...\n')
clean_train_reviews = []

for i in range(num_reviews):
    clean_train_reviews.append(review_to_words(train['review'][i]))
    sys.stdout.write('\rProgress: {:.2%}'.format((i/num_reviews)))

Cleaning and parsing the training set movie reviews...
Progress: 100.00%

### Create bag of words

In [19]:
from collections import Counter

In [20]:
vocab = Counter()

In [21]:
for review in clean_train_reviews:
    for word in review.split():
        vocab[word] += 1

In [22]:
vocab.most_common(5000)

[('movie', 44031),
 ('film', 40147),
 ('one', 26788),
 ('like', 20274),
 ('good', 15140),
 ('time', 12724),
 ('even', 12646),
 ('would', 12436),
 ('story', 11983),
 ('really', 11736),
 ('see', 11475),
 ('well', 10662),
 ('much', 9765),
 ('get', 9310),
 ('bad', 9301),
 ('people', 9285),
 ('also', 9156),
 ('first', 9061),
 ('great', 9058),
 ('made', 8362),
 ('way', 8026),
 ('make', 8023),
 ('could', 7922),
 ('movies', 7663),
 ('think', 7296),
 ('characters', 7154),
 ('character', 7023),
 ('watch', 6973),
 ('two', 6906),
 ('films', 6887),
 ('seen', 6679),
 ('many', 6675),
 ('life', 6628),
 ('plot', 6585),
 ('acting', 6490),
 ('never', 6484),
 ('love', 6454),
 ('little', 6435),
 ('best', 6416),
 ('show', 6295),
 ('know', 6166),
 ('ever', 5995),
 ('man', 5982),
 ('better', 5737),
 ('end', 5648),
 ('still', 5623),
 ('say', 5395),
 ('scene', 5378),
 ('scenes', 5207),
 ('go', 5157),
 ('something', 5077),
 ('back', 4972),
 ('real', 4737),
 ('watching', 4603),
 ('though', 4566),
 ('thing', 4526)

In [23]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

Creating the bag of words...



In [25]:
# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

In [29]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print(vocab)

['abandoned', 'abc', 'abilities', 'ability', 'able', 'abraham', 'absence', 'absent', 'absolute', 'absolutely', 'absurd', 'abuse', 'abusive', 'abysmal', 'academy', 'accent', 'accents', 'accept', 'acceptable', 'accepted', 'access', 'accident', 'accidentally', 'accompanied', 'accomplished', 'according', 'account', 'accuracy', 'accurate', 'accused', 'achieve', 'achieved', 'achievement', 'acid', 'across', 'act', 'acted', 'acting', 'action', 'actions', 'activities', 'actor', 'actors', 'actress', 'actresses', 'acts', 'actual', 'actually', 'ad', 'adam', 'adams', 'adaptation', 'adaptations', 'adapted', 'add', 'added', 'adding', 'addition', 'adds', 'adequate', 'admire', 'admit', 'admittedly', 'adorable', 'adult', 'adults', 'advance', 'advanced', 'advantage', 'adventure', 'adventures', 'advertising', 'advice', 'advise', 'affair', 'affect', 'affected', 'afford', 'aforementioned', 'afraid', 'africa', 'african', 'afternoon', 'afterwards', 'age', 'aged', 'agent', 'agents', 'ages', 'aging', 'ago', 'ag

In [64]:
### Cont frequency
import numpy as np

# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    #print(count, tag)
    if count == 44031:
        print(tag)

movie


### Train with random forest

In [34]:
print("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["sentiment"] )

Training the random forest...


### Creating a submission

In [36]:
# Read test data
test = pd.read_csv("testData.tsv", header=0, delimiter="\t", \
                   quoting=3 )

# Verify that there are 25,000 rows and 2 columns
print(test.shape)

(25000, 2)


In [43]:
# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])
clean_test_reviews = [] 

print("Cleaning and parsing the test set movie reviews...\n")
for i in range(0,num_reviews):
    clean_review = review_to_words( test["review"][i] )
    sys.stdout.write('\rProgress: {:.2%}'.format((i/num_reviews)))
    clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

Cleaning and parsing the test set movie reviews...

Progress: 100.00%

In [44]:
# # Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

In [49]:
# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )